### Notebook to view and add data. Also for testing scripts  

In [42]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=zmckee host=cs425.cc0vhnbhtoib.us-east-2.rds.amazonaws.com ")


In [97]:
cursor = conn.cursor()

In [43]:
%load_ext sql
%sql postgresql://zmckee:K?JQRD5<G>TV8CX:A8;G@cs425.cc0vhnbhtoib.us-east-2.rds.amazonaws.com/postgres

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: zmckee@postgres_1'

In [39]:
%%sql

SELECT table_name FROM information_schema.tables WHERE table_schema='public'

8 rows affected.


table_name
booking
booking_flight
airline
customer
flight
airport
address
creditcard


In [92]:
%%sql

DROP TABLE booking_flight;

CREATE TABLE booking_flight(
    bookingID INTEGER,
    airline_code VARCHAR(2),
    flight_number NUMERIC,
    flight_date	 DATE,
    seats NUMERIC, 
    FOREIGN KEY (bookingID) REFERENCES booking (id),
    FOREIGN KEY	(airline_code,flight_number,flight_date) REFERENCES flight(airline_code,flight_number,flight_date)
);

Done.
Done.


[]

In [44]:
%%sql

CREATE TABLE booking(
    id SERIAL NOT NULL PRIMARY KEY,
    class VARCHAR,
    paymentCard NUMERIC,
    user_ VARCHAR,
    FOREIGN KEY (user_) REFERENCES customer (email), 
    FOREIGN KEY (paymentCard) REFERENCES creditCard(number)
);



CREATE TABLE booking_flight(
    bookingID INTEGER,
    airline_code VARCHAR(2),
    flight_number NUMERIC,
    flight_date	 DATE,
    FOREIGN KEY (bookingID) REFERENCES booking (id),
    FOREIGN KEY	(airline_code,flight_number,flight_date) REFERENCES flight(airline_code,flight_number,flight_date)
);

(psycopg2.ProgrammingError) relation "booking" already exists
 [SQL: 'CREATE TABLE booking(\n    id SERIAL NOT NULL PRIMARY KEY,\n    class VARCHAR,\n    paymentCard NUMERIC,\n    user_ VARCHAR,\n    FOREIGN KEY (user_) REFERENCES customer (email), \n    FOREIGN KEY (paymentCard) REFERENCES creditCard(number)\n);']


In [93]:
%%sql

INSERT INTO flight VALUES 
('AA', 101, '2017-12-20', 'JFK', 'LAX', '03:00:00', '9:25:00', 15, 250, 600, 1200), 
('AA', 102, '2017-12-20', 'JFK', 'ORD', '03:00:00', '5:15:00', 15, 250, 100, 200), 
('AA', 103, '2017-12-20', 'JFK', 'DEN', '03:00:00', '7:00:00', 15, 250, 300, 600), 
('AA', 104, '2017-12-20', 'ORD', 'LAX', '06:00:00', '10:30:00', 15, 250, 300, 600), 
('AA', 105, '2017-12-20', 'ORD', 'DEN', '06:00:00', '8:45:00', 15, 250, 100, 200), 
('AA', 106, '2017-12-20', 'DEN', 'LAX', '09:30:00', '12:15:00', 15, 250, 100, 200)

6 rows affected.


[]

In [99]:
dates = ['2017-12-31'.format(x) for x in range(1,32)]

In [117]:
%%sql
INSERT INTO flight VALUES 
('AA', 201, '2017-12-31', 'LAX', 'JFK', '03:00:00', '9:25:00', 15, 250, 600, 1200), 
('AA', 202, '2017-12-31', 'ORD', 'JFK', '10:30:00', '12:45:00', 15, 250, 100, 200), 
('AA', 203, '2017-12-31', 'DEN', 'JFK', '07:00:00', '11:00:00', 15, 250, 300, 600), 
('AA', 204, '2017-12-31', 'LAX', 'ORD', '03:00:00', '07:30:00', 15, 250, 300, 600), 
('AA', 205, '2017-12-31', 'DEN', 'ORD', '07:00:00', '09:45:00', 15, 250, 100, 200), 
('AA', 206, '2017-12-31', 'LAX', 'DEN', '03:00:00', '06:15:00', 15, 250, 100, 200)

6 rows affected.


[]

In [123]:
%%sql

WITH RECURSIVE connections(f_codes, frm, dest, hops, price, airtime, arrival) AS (
  SELECT    airline_code || flight_number::text AS f_codes,
            depart_loc, 
            dest_loc,
            1 AS hops, 
            fc_price AS price,
            (arrival_time - depart_time) AS airtime,
            arrival_time
  FROM flight f0
  WHERE depart_loc = 'JFK' AND flight_date = '2017-12-20' 
  UNION ALL
  SELECT con.f_codes || '-' || f1.airline_code || f1.flight_number::text as flight
            , con.frm
            , f1.dest_loc
            , con.hops + 1 AS hops
            , con.price + f1.fc_price as price
            , (con.airtime + (f1.arrival_time - f1.depart_time)) AS total_time
            , f1.arrival_time
  FROM connections con
     JOIN flight f1
       ON f1.depart_loc = con.dest AND f1.flight_date = '2017-12-20'
       AND f1.depart_time > con.arrival 
)
SELECT *
FROM connections
WHERE dest = 'LAX' 



4 rows affected.


f_codes,frm,dest,hops,price,airtime,arrival
AA101,JFK,LAX,1,600,6:25:00,09:25:00
AA102-AA104,JFK,LAX,2,400,6:45:00,10:30:00
AA103-AA106,JFK,LAX,2,400,6:45:00,12:15:00
AA102-AA105-AA106,JFK,LAX,3,300,7:45:00,12:15:00


In [120]:
%%sql
SELECT (arrival_time - depart_time) AS airtime FROM flight

21 rows affected.


airtime
6:25:00
2:15:00
4:00:00
4:30:00
2:45:00
3:15:00
3:00:00
1:57:00
0:50:00
6:25:00


In [118]:
%%sql
SELECT * from flight

21 rows affected.


airline_code,flight_number,flight_date,depart_loc,dest_loc,depart_time,arrival_time,fc_capacity,econ_capacity,fc_price,econ_price
AA,201,2017-12-31,LAX,JFK,03:00:00,09:25:00,15,250,600,1200
AA,202,2017-12-31,ORD,JFK,10:30:00,12:45:00,15,250,100,200
AA,203,2017-12-31,DEN,JFK,07:00:00,11:00:00,15,250,300,600
AA,204,2017-12-31,LAX,ORD,03:00:00,07:30:00,15,250,300,600
AA,205,2017-12-31,DEN,ORD,07:00:00,09:45:00,15,250,100,200
AA,206,2017-12-31,LAX,DEN,03:00:00,06:15:00,15,250,100,200
AA,1,2017-10-31,ORD,SEA,05:00:00,08:00:00,20,160,5000,280
AA,1330,2017-12-26,ORD,CLT,18:26:00,20:23:00,None,144,251,113
AA,5709,2017-12-27,CLT,EWN,00:05:00,00:55:00,9,56,200,105
AA,101,2017-12-31,JFK,LAX,03:00:00,09:25:00,15,250,600,1200


In [ ]:
%%sql

In [ ]:
%%sql

In [ ]:
%%sql

In [ ]:
%%sql

In [ ]:
%%sql